In [1]:
!pip install -q -U google-genai

In [7]:
import google.generativeai as genai
import os

# ✅ Set API key globally using configure()
genai.configure(api_key="api key")

# ✅ Create model instance
model = genai.GenerativeModel(model_name="gemini-1.5-pro")

# Define the function to generate VB.NET code from DSL
def generate_code(input):
    prompt = f"""{dsl_syntax}

        {few_shot_examples}

        Now convert the following DSL code to VB.NET:
        DSL:
        {input}

        VB.NET:
    """
    # Send to Gemini
    response = model.generate_content([{"role": "user", "parts": [prompt]}])

    return response.text

# Define the DSL syntax and examples
dsl_syntax = """
You are a DSL-to-VB.NET code translator that returns only equivalent high quality VB.NET code when the input is DSL otherwise just returns 'Cannot generate VB.NET code for the following. Could not identify DSL code' .

Here is the DSL syntax you must follow:

--=========================================================================== 
-- This source code is part of MIGRATION
-- (C) Copyright Honeywell Inc. 2022.  All Rights Reserved. 
--=========================================================================== 
--
--
BLOCK UCNP2PLD (POINT UCNP2PLD; AT BACKGRND(1)) 
  
PARAMETER C6 : NUMBER                      -- UCN Number
PARAMETER C2 : STRING                      -- LCN directory for files 
PARAMETER C5 : STRING                      -- FULL LCN filename 
PARAMETER C4 : STRING                      -- Schematic Messages 
PARAMETER C3 : STRING                      -- Title string 
PARAMETER C7 : NUMBER                      -- progress indicator
  
------------------------------------------------------------------------
  
LOCAL g_File_Path, g_File_Name, g_File_Dir: STRING 
LOCAL g_Status, g_Fmgr_Sts, g_File_ID, g_Rec_Outs, g_Out_Priv : NUMBER 
LOCAL g_Filexist : LOGICAL
LOCAL g_FNStat: $MODSTR 
LOCAL g_title: STRING 
LOCAL g_DA_Stat, g_char_out, g_rec_out, g_node_no, g_out_buf: NUMBER
LOCAL g_Format_1, g_Format_2, g_Format_3, g_blanks: STRING
LOCAL g_Delimitr, g_Delimitrb, g_Delimitrc : STRING 
LOCAL g_node_type: $UCNNDTY 
LOCAL g_Format_4, g_Format_5 : STRING 
LOCAL g_index_no: NUMBER
LOCAL g_mp_status : CLERRSTS
LOCAL g_outer_idx, g_inner_idx, g_value : NUMBER
LOCAL g_sm_type : $SMPLTFM
LOCAL g_nts_stat : $CONVRS
LOCAL g_ucn_number, g_hwr, g_ehpm_node, g_sm_node : STRING
LOCAL g_c300pm_node : STRING
LOCAL g_ehpmx_node, g_fwr : STRING  
LOCAL g_offset : NUMBER
LOCAL g_time_fmt, g_time_delim : STRING 
LOCAL g_t : TIME
  
------------------------------------------------------------------------
%INCLUDE_SET FILE 
--
--
-- Initialize the LOCAL variables 
--
set g_Rec_Out = 1
set g_Out_Buf = 180
set g_Delimitrb = " "
set g_Delimitrc = ","
set g_File_Dir = C2
set g_Title = C3
set g_Out_Priv = 0
set g_blanks = "   "
set g_ehpm_node = "EHPM"
set g_sm_node = "TRI "
set g_c300pm_node = "CHPM"
set g_ehpmx_node = "XHPM"
  
--
-- set THE PROGRAM ON INDICATOR 
--
   set C4  = "P2P REPORT RUNNING" 
--
-- Initialize the file name and directory for future file handling requests.
--
   set g_File_Path = C2
   set g_File_Name = "UCN??P2P.XX"
  
-- Add the UCN Number to the file name
   CALL NUMBER_TO_STRING(g_nts_stat,g_ucn_number,C6,"R99") 
   CALL MODIFY_STRING(g_FNStat,g_File_Name,4,2,g_ucn_number,1)
  
-- Need to put file name in C5 parameter for schematic access
   set C5 = g_File_Name
  
-- Add the file name to the pathname
   CALL MODIFY_STRING(g_FNStat,g_File_Path,(LEN(g_File_Dir)+1), 
&      LEN(g_File_Name),g_file_Name,1)
  
--
-- Check if the output file exists for this UCN. If it does, delete the file. 
--
   CALL FILE$EXISTS(g_Status, g_Fmgr_Sts, g_File_Path, g_Filexist)
   IF g_Filexist THEN 
&       CALL FILE$DELETE_FILE(g_Status, g_Fmgr_Sts, g_File_Path)
  
--
-- Create the output file for this UCN. 
--
  
   CALL FILE$CREATE_FILE(g_Status, g_Fmgr_Sts, g_Out_Buf, g_File_Path)
   IF g_Status <> 0 THEN GOTO ERR1 
  
--
-- Open the output file. 
--
  
   CALL FILE$OPEN_FILE(g_Status, g_Fmgr_Sts, g_File_ID, g_Rec_Outs, 
&                      g_Out_Buf, g_File_Path, g_Out_Priv)
   IF g_Status <> 0 THEN GOTO ERR2 
  
--
-- Write the title line. 
--
   set g_Char_Out = 1
   set g_Format_1 = "TEXTL1:45" 
   CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID,g_Title,
&                      g_Format_1,g_Delimitrb) 
   CALL FILE$WRITE_RECORD(g_Status, g_Fmgr_Sts, g_File_ID, g_Rec_Out) 
   set g_Rec_Out = g_Rec_Out + 1 
  
--
-- Write the Date and Time line.
--
  
   SET g_time_fmt = "DATEDD AM:3 YYENDDATE" 
   SET g_time_delim = " " 
   SET g_offset = 1            -- Set pointer to start of record
   SET g_t = date_time         -- Get current date in g_t
   CALL File$Put_Field (g_status, g_da_stat, g_offset, g_File_ID, g_t,
&                         g_time_fmt, g_time_delim) 
  
   SET g_time_fmt = "TIMEHH:MM:SSENDTIME" 
   SET g_t = now               -- Get current time in t
   CALL File$Put_Field (g_status, g_da_stat, g_offset, g_File_ID, g_t,
&                         g_time_fmt, g_time_delim) 
   SET g_offset = 1            -- Reset pointer to start of record
  
   CALL FILE$WRITE_RECORD(g_Status, g_Fmgr_Sts, g_File_ID, g_Rec_Out) 
   set g_Rec_Out = g_Rec_Out + 1 
  
--
-- Write the node types title line. 
--
   set g_Char_Out = 1
   set g_Format_4 = "TEXTL1:4" 
   set g_index_no = 1
   CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID,g_blanks,
&                      g_Format_4,g_Delimitrc) 
TLOOP1: CALL MOVE_PARAMETER (g_node_type,  C1(g_index_no).NODETYP,  
&                            g_mp_status)
  
-- Check for errors in node type or unconfigured
   IF g_mp_status <> NOERROR THEN GOTO TLOOP1a
   ELSE IF g_node_type = NOTCONFG THEN GOTO TLOOP1a 
  
-- IF an HPM, get the TBC revision
   set g_hwr = "   "
   set g_fwr = "   "
   IF g_node_type = HPM THEN CALL MOVE_PARAMETER (g_hwr,  
&                            C1(g_index_no).UTSTBCRV,  
&                            g_mp_status)
-- Save way to tell either C300PM or EHPMX
   IF g_node_type = HPM THEN CALL MOVE_PARAMETER (g_fwr,  
&                            C1(g_index_no).COMFWREV,  
&                            g_mp_status)
   IF (g_node_type = HPM) AND (g_hwr >= "P") THEN GOTO Tloop1b
  
-- IF an SM, get the SM type
   IF g_node_type = SM THEN (CALL MOVE_PARAMETER (g_sm_type,  
&                           C1(g_index_no).SMPLTFM,  
&                           g_mp_status); GOTO TLOOP1c)
  
-- Output returned node type  
   IF g_index_no < 32 THEN set g_Delimitr = g_Delimitrc
   ELSE set g_Delimitr = g_Delimitrb
   CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID,
&                      g_node_type,g_Format_4,g_Delimitr) 
GOTO TLOOP1d
 
-- Output blank node type  
TLOOP1a: IF g_index_no < 32 THEN set g_Delimitr = g_Delimitrc
   ELSE set g_Delimitr = g_Delimitrb
   CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID,
&                      g_blanks,g_Format_4,g_Delimitr) 
GOTO TLOOP1d
  
-- Output EHPM or C300PM or EHPMX node type
TLOOP1b: IF g_index_no < 32 THEN set g_Delimitr = g_Delimitrc
   ELSE set g_Delimitr = g_Delimitrb
-- Test for EHPM: TBC revision of "P" is an EHPM
   IF (g_hwr = "P") THEN
&  CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID, 
&                      g_ehpm_node,g_Format_4,g_Delimitr) 
-- Test for C300PM: TBC revision of "Q" is a C300PM 
   IF ((g_hwr = "Q") AND (g_fwr <> "DD")) THEN
&  CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID, 
&                      g_c300pm_node,g_Format_4,g_Delimitr)
-- Test for EHPMX: TBC revision of "Q" and FWREV of "DD" is an EHPMX 
   ELSE IF ((g_hwr = "Q") AND (g_fwr = "DD")) THEN
&  CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID, 
&                      g_ehpmx_node,g_Format_4,g_Delimitr)
 
GOTO TLOOP1d
  
-- Output SM node type  
TLOOP1c: IF g_index_no < 32 THEN set g_Delimitr = g_Delimitrc
   ELSE set g_Delimitr = g_Delimitrb
   IF g_sm_type = FSC THEN set g_sm_node = "FSC "
   ELSE set g_sm_node = "TRI "
   CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID,
&                      g_sm_node,g_Format_4,g_Delimitr) 
  
TLOOP1d: set g_index_no = g_index_no + 1
   IF g_index_no <= 32 THEN GOTO TLOOP1 
   CALL FILE$WRITE_RECORD(g_Status, g_Fmgr_Sts, g_File_ID, g_Rec_Out) 
   set g_Rec_Out = g_Rec_Out + 1 
  
--
-- Write the node numbers title line. 
--
   set g_Char_Out = 1
   set g_Format_2 = "TEXTL1:2"
   CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID,g_blanks,
&                      g_Format_2,g_Delimitrc) 
  
   set g_node_no = 1
   set g_Format_3 = "R99" 
TLOOP2: IF g_node_no < 63 THEN set g_Delimitr = g_Delimitrc 
   ELSE set g_Delimitr = g_Delimitrb 
   CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID,
&                      g_node_no,g_Format_3,g_Delimitr) 
   set g_node_no = g_node_no + 2
   IF g_node_no < 64 THEN GOTO TLOOP2 
   CALL FILE$WRITE_RECORD(g_Status, g_Fmgr_Sts, g_File_ID, g_Rec_Out) 
   set g_Rec_Out = g_Rec_Out + 1 
  
-- BEGIN Outer loop
   Set g_outer_idx = 1
  
OLoop: set g_Char_Out = 1
  
-- Report Progress
   Set C7 = (g_outer_idx * 2) -1
  
   call BKG_DELAY (0 SECS);  -- task switch 
  
-- First, get node type
   set g_Format_4 = "TEXTL1:4" 
   CALL MOVE_PARAMETER (g_node_type,  C1(g_outer_idx).NODETYP,  
&                            g_mp_status)
  
-- IF an SM, get the SM type
   IF g_node_type = SM THEN CALL MOVE_PARAMETER (g_sm_type,  
&                           C1(g_outer_idx).SMPLTFM,  
&                           g_mp_status)
  
-- Check for errors in node type or unconfigured/unsupported node types
   IF g_mp_status <> NOERROR THEN GOTO End_Oloop
   ELSE IF g_node_type = NOTCONFG THEN GOTO End_Oloop 
   ELSE IF g_node_type = NIM THEN GOTO End_Oloop 
   ELSE IF g_node_type = LM THEN GOTO End_Oloop 
  
-- IF an HPM, get the TBC revision
   set g_hwr = "   "
   set g_fwr = "   "
   IF g_node_type = HPM THEN CALL MOVE_PARAMETER (g_hwr,  
&                            C1(g_outer_idx).UTSTBCRV,  
&                            g_mp_status)
-- Save way to tell either C300PM or EHPMX
   IF g_node_type = HPM THEN CALL MOVE_PARAMETER (g_fwr,  
&                            C1(g_outer_idx).COMFWREV,  
&                            g_mp_status)
  
-- Node is a valid type and configured
-- Put node type in first field of NPRQUAVG line 
   IF (g_node_type = SM) THEN GOTO OLOOP1 
-- Test for EHPM and C300PM node type 
--     TBC revision of "P" is an EHPM 
--     TBC revision of "Q" is a C300PM
--     TBC revision of "Q" and FWREV of "DD" is an EHPMX
   IF (g_node_type = HPM) AND (g_hwr = "P") THEN
&  CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID,
&                      g_ehpm_node,g_Format_4,g_Delimitrc) 
   ELSE IF ((g_node_type = HPM) AND (g_hwr = "Q") AND (g_fwr <> "DD"))  THEN 
&  CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID, 
&                      g_c300pm_node,g_Format_4,g_Delimitrc)
   ELSE IF ((g_node_type = HPM) AND (g_hwr = "Q") AND (g_fwr = "DD"))  THEN 
&  CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID, 
&                      g_ehpmx_node,g_Format_4,g_Delimitrc)
   ELSE CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID,
&                      g_node_type,g_Format_4,g_Delimitrc) 
   GOTO OLOOP2
  
OLOOP1: CALL MOVE_PARAMETER (g_sm_type,  
&                           C1(g_outer_idx).SMPLTFM,  
&                           g_mp_status)
 
   IF g_sm_type = FSC THEN set g_sm_node = "FSC "
   ELSE set g_sm_node = "TRI "
 
   CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID,
&                      g_sm_node,g_Format_4,g_Delimitrc) 
  
-- BEGIN Inner loop for parameter NPRQUAVG
OLOOP2: Set g_inner_idx = 1
   set g_Format_5 = "RZZZ9" 
   set g_value = 0  -- init 
ILoop1: CALL MOVE_PARAMETER (g_value,  
&       C1(g_outer_idx).NPRQUAVG(g_inner_idx), g_mp_status)
  
-- Check for errors in NPRQUAVG
   IF g_mp_status <> NOERROR THEN set g_value = 0
  
   IF g_inner_idx < 62 THEN set g_Delimitr = g_Delimitrc
   ELSE set g_Delimitr = g_Delimitrb
  
   CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID,
&                      g_value,g_Format_5,g_Delimitr) 
  
   Set g_inner_idx = g_inner_idx + 2
   IF g_inner_idx < 64 THEN GOTO ILoop1
-- END Inner loop for parameter NPRQUAVG
-- write line of NPRQUAVG values
   CALL FILE$WRITE_RECORD(g_Status, g_Fmgr_Sts, g_File_ID, g_Rec_Out) 
   set g_Rec_Out = g_Rec_Out + 1 
  
   set g_Char_Out = 1
-- Put node number in first field of NPRSPAVG line  
   set g_node_no = (g_outer_idx*2)-1
   CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID,
&                      g_node_no,g_Format_3,g_Delimitrc) 
  
-- BEGIN Inner loop for parameter NPRSPAVG
   Set g_inner_idx = 1
   set g_Format_5 = "RZZZ9" 
   set g_value = 0  -- init 
ILoop2: CALL MOVE_PARAMETER (g_value,  
&       C1(g_outer_idx).NPRSPAVG(g_inner_idx), g_mp_status)
  
-- Check for errors in NPRSPAVG
   IF g_mp_status <> NOERROR THEN set g_value = 0
  
   IF g_inner_idx < 62 THEN set g_Delimitr = g_Delimitrc
   ELSE set g_Delimitr = g_Delimitrb
  
   CALL FILE$PUT_FIELD(g_Status,g_DA_Stat,g_Char_Out,g_File_ID,
&                      g_value,g_Format_5,g_Delimitr) 
  
   Set g_inner_idx = g_inner_idx + 2
   IF g_inner_idx < 64 THEN GOTO ILoop2
-- END Inner loop for parameter NPRSPAVG
-- write line of NPRSPAVG values
   CALL FILE$WRITE_RECORD(g_Status, g_Fmgr_Sts, g_File_ID, g_Rec_Out) 
   set g_Rec_Out = g_Rec_Out + 1 
  
End_Oloop: Set g_outer_idx = g_outer_idx + 1
   IF g_outer_idx <= 32 THEN GOTO OLoop
-- END Outer loop
  
   GOTO XEXIT
--
-- HANDLE ERROR CONDITIONS
--
  
ERR1: set C4 = "ERROR CREATING FILE" 
   GOTO XEXIT
  
ERR2: set C4 = "ERROR OPENING FILE" 
   GOTO XEXIT
  
--
-- Close the file.
--
XEXIT:   CALL FILE$CLOSE_FILE(g_Status, g_Fmgr_Sts, g_File_ID)
 
-- check the Close_File status and presence of the other 
-- two errors before sending the complete message.
 
If ((g_status = 0) AND ((C4 <> "ERROR CREATING FILE")
&    OR (C4 <> "ERROR OPENING FILE"))) 
& THEN set C4  = "P2P REPORT COMPLETE"
 
   EXIT
--
END UCNP2PLD
`
"""

few_shot_examples = """
Example 1:
DSL:
SET TEMP = 100
CALL StartHeater(TEMP)

VB.NET:
Dim TEMP As Integer = 100
StartHeater(TEMP)

Example 2:
DSL:
IF TEMP > 80 THEN
  CALL ActivateCooling(TEMP)
ELSE
  CALL Idle()
END BLOCK

VB.NET:
If TEMP > 80 Then
  ActivateCooling(TEMP)
Else
  Idle()
End If

example 3:
Python:
class Vehicle:
    def __init__(self, type):
        self.type = type

    def describe(self):
        print(f"This is a {self.type}")

car = Vehicle("car")
car.describe()

VB.NET:
Cannot generate VB.NET code for the following. Could not identify DSL code
"""

res = generate_code("""BLOCK CONFIG_NODES (POINT CONFIG; AT BACKGRND(1)) 
PARAMETER C1 : NUMBER  -- UCN Number
LOCAL g_node_type: $UCNNDTY
LOCAL g_mp_status: CLERRSTS

LOOP: FOR index = 1 TO 32
  CALL MOVE_PARAMETER(g_node_type, C1(index).NODETYP, g_mp_status)
  IF g_node_type = HPM THEN
    CALL SET_PARAMETER(C1(index).SCAN_RATE, 200)  -- 200ms for HPM
  ELSE IF g_node_type = SM THEN
    CALL SET_PARAMETER(C1(index).REDUNDANCY, TRIPLEX)  -- Safety modules
  END IF
NEXT index

END BLOCK CONFIG_NODES""")

print(res)

VB.NET:
```vb.net
Imports CLERRSTS ' Assuming CLERRSTS is a defined enum or class
Imports UCNNDTY ' Assuming UCNNDTY is a defined enum or class

' Assuming C1 is a defined array of structures with members like NODETYP, SCAN_RATE and REDUNDANCY
' and assuming SET_PARAMETER and MOVE_PARAMETER are defined functions/subs


Sub CONFIG_NODES() '  POINT CONFIG; AT BACKGRND(1) - Directives are not converted directly

    Dim g_node_type As UCNNDTY
    Dim g_mp_status As CLERRSTS
    Dim index As Integer

    For index = 1 To 32
        MOVE_PARAMETER(g_node_type, C1(index).NODETYP, g_mp_status)
        If g_node_type = HPM Then
            SET_PARAMETER(C1(index).SCAN_RATE, 200)  ' 200ms for HPM
        ElseIf g_node_type = SM Then
            SET_PARAMETER(C1(index).REDUNDANCY, TRIPLEX)  ' Safety modules
        End If
    Next

End Sub
```



In [ ]:
pip install google-generativeai

In [ ]:
!pip show google-generativeai